In [64]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# train, test, validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix

# models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


# preprocessing
from sklearn.preprocessing import MinMaxScaler


In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
combine = [train_df, test_df]

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# del Ticket, Cabin columns
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)
combine = [train_df, test_df]

In [5]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


# add title

In [6]:
# add title

for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
train_df.Title.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Capt          1
Lady          1
Don           1
Sir           1
Ms            1
Countess      1
Jonkheer      1
Mme           1
Name: Title, dtype: int64

# del rare title and map value

## todo: try one hote encoding without delete rare title

In [7]:
# del rare title and map value
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                                                 'Don', 'Dr', 'Major', 'Rev', 'Sir',
                                                 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1


# drop Name, PassengerId

In [8]:
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,male,22.0,1,0,7.2500,S,1
1,1,1,female,38.0,1,0,71.2833,C,3
2,1,3,female,26.0,0,0,7.9250,S,2
3,1,1,female,35.0,1,0,53.1000,S,3
4,0,3,male,35.0,0,0,8.0500,S,1


# map value to Sex 

In [9]:
for dataset in combine:
    dataset["Sex"] = dataset["Sex"].map({'female':1, 'male':0}).astype(int)
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22.0,1,0,7.2500,S,1
1,1,1,1,38.0,1,0,71.2833,C,3
2,1,3,1,26.0,0,0,7.9250,S,2
3,1,1,1,35.0,1,0,53.1000,S,3
4,0,3,0,35.0,0,0,8.0500,S,1


# fill na value

In [10]:
train_df.isna().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
Embarked     True
Title       False
dtype: bool

In [11]:
test_df.isna().any()

PassengerId    False
Pclass         False
Sex            False
Age             True
SibSp          False
Parch          False
Fare            True
Embarked       False
Title          False
dtype: bool

## fill na of Age

In [12]:
guess_ages = np.zeros((2,3))

for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,22,1,0,7.2500,S,1
1,1,1,1,38,1,0,71.2833,C,3
2,1,3,1,26,0,0,7.9250,S,2
3,1,1,1,35,1,0,53.1000,S,3
4,0,3,0,35,0,0,8.0500,S,1


In [13]:
train_df.isna().any()

Survived    False
Pclass      False
Sex         False
Age         False
SibSp       False
Parch       False
Fare        False
Embarked     True
Title       False
dtype: bool

## add age band

In [14]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,AgeBand
0,0,3,0,22,1,0,7.2500,S,1,"(16.0, 32.0]"
1,1,1,1,38,1,0,71.2833,C,3,"(32.0, 48.0]"
2,1,3,1,26,0,0,7.9250,S,2,"(16.0, 32.0]"
3,1,1,1,35,1,0,53.1000,S,3,"(32.0, 48.0]"
4,0,3,0,35,0,0,8.0500,S,1,"(32.0, 48.0]"


## Overwrite AgeBand number on Age. means, drop Age and AgeBand text column

### todo: keep Age feature and add AgeBand numerical feature

In [15]:

for dataset in combine:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title
0,0,3,0,1,1,0,7.2500,S,1
1,1,1,1,2,1,0,71.2833,C,3
2,1,3,1,1,0,0,7.9250,S,2
3,1,1,1,2,1,0,53.1000,S,3
4,0,3,0,2,0,0,8.0500,S,1


# Create new feature

In [16]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass
    

train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,FamilySize,IsAlone,Age*Class
0,0,3,0,1,1,0,7.2500,S,1,2,0,3
1,1,1,1,2,1,0,71.2833,C,3,2,0,2
2,1,3,1,1,0,0,7.9250,S,2,1,1,3
3,1,1,1,2,1,0,53.1000,S,3,2,0,2
4,0,3,0,2,0,0,8.0500,S,1,1,1,6


# drop Parch, SibSp, FaimilySize

## todo : think more about these features

In [17]:
# drop Parch, SibSp, FaimilySize

train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,7.2500,S,1,0,3
1,1,1,1,2,71.2833,C,3,0,2
2,1,3,1,1,7.9250,S,2,1,3
3,1,1,1,2,53.1000,S,3,0,2
4,0,3,0,2,8.0500,S,1,1,6


# fill missing Embarked 

In [18]:
freq_port = train_df.Embarked.dropna().mode()[0]
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)


# Converting Embarked categorical feature to numeric

## todo : try one hot encoding

In [19]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,7.2500,0,1,0,3
1,1,1,1,2,71.2833,1,3,0,2
2,1,3,1,1,7.9250,0,2,1,3
3,1,1,1,2,53.1000,0,3,0,2
4,0,3,0,2,8.0500,0,1,1,6


# fill na of test data Fare

In [20]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,892,3,0,2,7.8292,2,1,1,6
1,893,3,1,2,7.0000,0,3,0,6
2,894,2,0,3,9.6875,2,1,1,6
3,895,3,0,1,8.6625,0,1,1,3
4,896,3,1,1,12.2875,0,3,0,3


# make Fareband feature

## todo : try more fare band number
## todo : keep Fare feature and add FareBand numerical feature¶

In [21]:
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)

for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)

combine = [train_df, test_df]
    
train_df.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
0,0,3,0,1,0,0,1,0,3
1,1,1,1,2,3,1,3,0,2
2,1,3,1,1,1,0,2,1,3
3,1,1,1,2,3,0,3,0,2
4,0,3,0,2,1,0,1,1,6
5,0,3,0,1,1,2,1,1,3
6,0,1,0,3,3,0,1,1,3
7,0,3,0,0,2,0,4,0,0
8,1,3,1,1,1,0,3,0,3
9,1,2,1,0,2,1,3,0,0


----

# model and estimate

## try several models

In [22]:
X_train_df = train_df.drop("Survived", axis=1)
y_train_df = train_df["Survived"]

X_test_df = test_df.drop("PassengerId", axis=1).copy()

In [23]:
logreg = LogisticRegression()
scores = cross_val_score(logreg, X_train_df, y_train_df)
scores.mean()

0.8069584736251403

In [24]:
scores

array([0.8013468 , 0.81818182, 0.8013468 ])

In [25]:
# cannnot get model attribute because of cross_val_score use
"""
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)
"""

'\ncoeff_df = pd.DataFrame(train_df.columns.delete(0))\ncoeff_df.columns = [\'Feature\']\ncoeff_df["Correlation"] = pd.Series(logreg.coef_[0])\n\ncoeff_df.sort_values(by=\'Correlation\', ascending=False)\n'

In [26]:
svc = SVC(C=10.0, kernel="rbf")
scores = cross_val_score(svc, X_train_df, y_train_df)
scores.mean()

0.8069584736251403

In [27]:
gaussianNb = GaussianNB()
scores = cross_val_score(gaussianNb, X_train_df, y_train_df)
scores.mean()

0.6408529741863075

In [28]:
perceptron = Perceptron()
scores = cross_val_score(perceptron, X_train_df, y_train_df)
scores.mean()

/home/yuki/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/yuki/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/yuki/.local/lib/python3.6/site-packages/skle

0.6408529741863075

In [29]:
linear_svc = LinearSVC()
scores = cross_val_score(linear_svc, X_train_df, y_train_df)
scores.mean()

0.7912457912457912

In [30]:
sgd = SGDClassifier()
scores = cross_val_score(sgd, X_train_df, y_train_df)
scores.mean()

/home/yuki/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/yuki/.local/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/home/yuki/.local/lib/pyth

0.7272727272727272

In [31]:
decision_tree = DecisionTreeClassifier()
scores = cross_val_score(decision_tree, X_train_df, y_train_df)
scores.mean()

0.7946127946127947

In [32]:
random_forest = RandomForestClassifier(n_estimators=100)
scores = cross_val_score(random_forest, X_train_df, y_train_df)
scores.mean()

0.8047138047138046

----

# make train, test data set from train.csv

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_train_df, y_train_df, test_size=0.33, random_state=42)
print("train/test data shape", X_train.shape, X_test.shape)

train/test data shape (596, 8) (295, 8)


In [34]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

#param_grid = [{'kernel': ['rbf'],
#               'C': [0.001, 0.01, 0.1, 1, 10, 100],
#               'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
#              {'kernel': ['linear'],
#               'C': [0.001, 0.01, 0.1, 1, 10, 100]}]

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
#grid_search.fit(X_train, y_train)

scores = cross_val_score(grid_search, X_train, y_train, cv=5)
print("train", scores.mean())

# cannot show grid_search attribute because using cross_val_score
#print(grid_search.score(X_train, y_train))
#print(grid_search.best_params_)


train 0.8003734035987276


# apply preprocessing

In [35]:
X_train.head()

,Pclass,Sex,Age,Fare,Embarked,Title,IsAlone,Age*Class
6,1,0,3,3,0,1,1,3
718,3,0,1,2,2,1,1,3
685,2,0,1,3,1,1,0,2
73,3,0,1,2,1,1,0,3
882,3,1,1,1,0,2,1,3


### Min Max Scaler

In [36]:
#minmaxScaler = MinMaxScaler()
#minmaxScaler.fit(X_train)

#X_minmaxScaled = minmaxScaler.transform(X)
#X_minmaxScaled[0]

In [37]:
# grid search SVC
# use cross_val_score

#param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
#              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

#param_grid = [{'kernel': ['rbf'],
#               'C': [0.001, 0.01, 0.1, 1, 10, 100],
#               'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
#              {'kernel': ['linear'],
#               'C': [0.001, 0.01, 0.1, 1, 10, 100]}]

#grid_search = GridSearchCV(SVC(), param_grid, cv=5)
#grid_search.fit(X_minmaxScaled, y)

#scores = cross_val_score(grid_search, X_minmaxScaled, y, cv=5)
#print("train", scores.mean())

In [38]:
pipe = make_pipeline(MinMaxScaler(), SVC())
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7966101694915254

In [77]:
# grid search for pipeline
param_grid = {'svc__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svc__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

pipe = make_pipeline(MinMaxScaler(), SVC())

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.score(X_test, y_test))
print(grid.best_params_)

0.8204697986577181
0.7898305084745763
{'svc__C': 10, 'svc__gamma': 1}


In [79]:
# watching wrong result

pipe.fit(X_train, y_train)
print("test", pipe.score(X_test, y_test))

scores = cross_val_score(pipe, X_train_df, y_train_df, cv=5)
print("cross eval for all", scores.mean())

y_predicted = pipe.predict(X_test)

confusion = confusion_matrix(y_test, y_predicted)
print("Confusion matrix:\n{}".format(confusion))

for i, (y, y_pred) in enumerate(zip(y_test, y_predicted)):
    if y != y_pred:
        print(X_test.iloc[i,:], "\n", y_pred, "should be", y)
        print()

test 0.7966101694915254
cross eval for all 0.7867150249291879
Confusion matrix:
[[151  24]
 [ 36  84]]
Pclass       3
Sex          0
Age          1
Fare         2
Embarked     1
Title        4
IsAlone      0
Age*Class    3
Name: 709, dtype: int64 
 0 should be 1

Pclass       3
Sex          1
Age          1
Fare         2
Embarked     0
Title        2
IsAlone      0
Age*Class    3
Name: 485, dtype: int64 
 1 should be 0

Pclass       1
Sex          0
Age          2
Fare         3
Embarked     0
Title        1
IsAlone      0
Age*Class    2
Name: 621, dtype: int64 
 0 should be 1

Pclass       1
Sex          0
Age          2
Fare         2
Embarked     0
Title        1
IsAlone      1
Age*Class    2
Name: 447, dtype: int64 
 0 should be 1

Pclass       2
Sex          0
Age          1
Fare         1
Embarked     0
Title        1
IsAlone      1
Age*Class    2
Name: 673, dtype: int64 
 0 should be 1

Pclass       3
Sex          1
Age          1
Fare         0
Embarked     0
Title        2
Is

In [40]:
# random forest result for compare
#random_forest = RandomForestClassifier(n_estimators=100)
#random_forest.fit(X_minmaxScaled, y)

#scores = cross_val_score(grid_search, X_minmaxScaled, y, cv=5)
#print("train", scores.mean())

pipe = make_pipeline(MinMaxScaler(), RandomForestClassifier(n_estimators=100))
pipe.fit(X_train, y_train)
print("test", pipe.score(X_test, y_test))

scores = cross_val_score(pipe, X_train_df, y_train_df, cv=5)
print("cross eval for all", scores.mean())

test 0.8
cross eval for all 0.8081898029886656


### watch wrong result

In [72]:
ranfore = pipe.steps[1][1]
ranfore

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [65]:
# watching wrong result
y_predicted = pipe.predict(X_test)

confusion = confusion_matrix(y_test, y_predicted)
print("Confusion matrix:\n{}".format(confusion))

for i, (y, y_pred) in enumerate(zip(y_test, y_predicted)):
    if y != y_pred:
        print(X_test.iloc[i,:], "\n", y_pred, "should be", y)
        print()

Confusion matrix:
[[146  29]
 [ 30  90]]
Pclass       3
Sex          0
Age          1
Fare         2
Embarked     1
Title        4
IsAlone      0
Age*Class    3
Name: 709, dtype: int64 
 0 should be 1

Pclass       1
Sex          0
Age          2
Fare         3
Embarked     0
Title        1
IsAlone      0
Age*Class    2
Name: 621, dtype: int64 
 0 should be 1

Pclass       3
Sex          1
Age          1
Fare         0
Embarked     0
Title        2
IsAlone      0
Age*Class    3
Name: 192, dtype: int64 
 0 should be 1

Pclass       3
Sex          0
Age          0
Fare         2
Embarked     0
Title        4
IsAlone      0
Age*Class    0
Name: 819, dtype: int64 
 1 should be 0

Pclass       2
Sex          0
Age          1
Fare         1
Embarked     0
Title        1
IsAlone      1
Age*Class    2
Name: 673, dtype: int64 
 0 should be 1

Pclass       3
Sex          0
Age          0
Fare         2
Embarked     0
Title        4
IsAlone      0
Age*Class    0
Name: 63, dtype: int64 
 1 should 

# todo

- [ ] try grid search
- [ ] other model
- [ ] make preprocessing
- [ ] think train/test/validation data
- 
